This program cuts the input images into square shapes ([64, 64], [48,48], [32,32]..), converts them to grayscale and then converts them to progment array

In [9]:
import os
import numpy as np
from PIL import Image

In [ ]:
input_folder = 'digit_dataset'
output_folder = 'digit_array'

# Create output base directory
os.makedirs(output_folder, exist_ok=True)

# List class folders
class_folders = [d for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]

for class_name in class_folders:
    input_class_folder = os.path.join(input_folder, class_name)
    output_class_folder = os.path.join(output_folder, class_name)
    os.makedirs(output_class_folder, exist_ok=True)

    image_files = [f for f in os.listdir(input_class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    for filename in image_files:
        try:
            # FIX: Use input_class_folder instead of input_folder
            img_path = os.path.join(input_class_folder, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img_resized = img.resize((32,32))
            img_array = np.asarray(img_resized, dtype=np.uint8)

            # Ensure correct shape
            if img_array.shape != (32,32):
                raise ValueError(f"Resized image shape incorrect: {img_array.shape}")

            flat_array = img_array.flatten()
            if flat_array.shape[0] != 32*32:
                raise ValueError(f"Flattened array length is not 4096: {flat_array.shape[0]}")

            # Format to C-style progmem array
            variable_name = os.path.splitext(filename)[0].replace(" ", "_").replace("-", "_")
            array_lines = ', '.join(str(v) for v in flat_array)
            c_array = f"const uint8_t {variable_name}_progmem[] = {{\n  {array_lines}\n}};\n"

            # Save to .txt
            output_path = os.path.join(output_class_folder, f"{variable_name}.txt")
            with open(output_path, 'w') as f:
                f.write(c_array)

            print(f"✅ {class_name}/{filename} -> {variable_name}.txt")

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print("\n✅ All images processed successfully.")